In [270]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime as dt
from nltk.tokenize import RegexpTokenizer

In [271]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [272]:
df = pd.read_csv('griddis.csv')

Let's look at our data and consider what we can do to clean it up some.

In [273]:
df.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
0,Severe Weather - Thunderstorms,2014,6/30/2014,8:00 PM,7/2/2014,6:30 PM,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm"
1,Severe Weather - Thunderstorms,2014,6/30/2014,11:20 PM,7/1/2014,5:00 PM,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm"
2,Severe Weather - Thunderstorms,2014,6/30/2014,5:55 PM,7/1/2014,2:53 AM,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm"
3,Fuel Supply Emergency - Coal,2014,6/27/2014,1:21 PM,Unknown,Unknown,We Energies,Wisconsin,MRO,Unknown,Unknown,"fuel supply emergency, coal"
4,Physical Attack - Vandalism,2014,6/24/2014,2:54 PM,6/24/2014,2:55 PM,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical"


There's quite a few issues here already, the Tags feature being the first to jump out. We'll need to sort these tags into features to look at them individually eventually. First let's look at all the missing or badly formatted data.

In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 12 columns):
Event Description               1652 non-null object
Year                            1652 non-null int64
Date Event Began                1652 non-null object
Time Event Began                1643 non-null object
Date of Restoration             1638 non-null object
Time of Restoration             1632 non-null object
Respondent                      1652 non-null object
Geographic Areas                1651 non-null object
NERC Region                     1650 non-null object
Demand Loss (MW)                1254 non-null object
Number of Customers Affected    1438 non-null object
Tags                            1651 non-null object
dtypes: int64(1), object(11)
memory usage: 155.0+ KB


Looks like Demand Loss and Number of Customers Affected might be problematic columns to analyse. First, let's look at the times and dates columns format and see which ones we can get into a sensible datetime format.

In [289]:
df[pd.isnull(pd.to_datetime(df['Time Event Began'], 'coerce'))]['Time Event Began']

1334    NaN
1396    NaN
1625    NaN
1629    NaN
1630    NaN
1638    NaN
1641    NaN
1643    NaN
1644    NaN
1645    NaN
Name: Time Event Began, dtype: object

There's not much we can do about the nans or Ongoing. However, the others we can probably fix. The '5:78 p.m.' we'll just change to '5:58 PM', as this is the most sense we can make of this time considering the issue was a voltage reduction. There's also an 'Evening' entry that we'll just drop, as there's only one of them and this time is too vague for us to make any sense of.

In [288]:
df[pd.isnull(pd.to_datetime(df['Time of Restoration'], 'coerce'))]['Time of Restoration']

3       NaN
15      NaN
25      NaN
27      NaN
36      NaN
46      NaN
86      NaN
89      NaN
102     NaN
104     NaN
254     NaN
369     NaN
986     NaN
1099    NaN
1196    NaN
1226    NaN
1334    NaN
1396    NaN
1398    NaN
1454    NaN
1590    NaN
1593    NaN
1601    NaN
1617    NaN
1623    NaN
1625    NaN
1629    NaN
1630    NaN
1632    NaN
1633    NaN
1635    NaN
1638    NaN
1639    NaN
1641    NaN
1643    NaN
1644    NaN
1645    NaN
1648    NaN
1650    NaN
Name: Time of Restoration, dtype: object

In [287]:
df.replace('Unknown', np.nan, inplace=True)
df.replace('Ongoing', np.nan, inplace=True)
df.replace('Midnight', '12:00 AM', inplace=True)
df.replace('12:00 noon', '12:00 PM', inplace=True)
df.replace('12 noon', '12:00 PM', inplace= True)
df.replace('3: 37 p.m.', '3:37 PM', inplace=True)
df.replace('12 a.m.', '12:00 PM', inplace=True)
df.replace('5:78 p.m.', '5:58 PM', inplace=True)
df.replace('9: 52 a.m.', '9:52 AM', inplace=True)
df = df[df['Time Event Began'] != 'Evening']

In [304]:
df["Time Event Began"] = pd.to_datetime(df["Time Event Began"], infer_datetime_format=True, errors='coerce').dt.time
df["Time of Restoration"] = pd.to_datetime(df["Time of Restoration"], infer_datetime_format=True, errors='coerce').dt.time

Next, let's look at the Tags column

In [305]:
df[pd.isnull(df['Tags'])]

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
1531,Wild Fire _ Transmission Equipment,2003,12/1/2003,NaT,12/1/2003,NaT,REMVEC,Cape Cod and part of SE Massachusetts,NPCC,630,"300,000",NaN


In [307]:
labels = []
tokenizer = RegexpTokenizer(r'\w+[\s\w]+')
tokenized = [tokenizer.tokenize(str(i)) for i in df['Tags']]
for j in tokenized:
    for k in j:
        labels.append(k)
print(set(labels))

{'physical', 'fog', 'unknown', 'hail', 'energy deficiency alert', 'hurricane', 'load shedding', 'coal', 'breaker trip', 'transmission interruption', 'distribution interruption', 'tropical storm', 'natural gas', 'winter storm', 'heat', 'earthquake', 'uncontrolled loss', 'wild fire', 'tornado', 'islanding', 'thunderstorm', 'cyber', 'cold', 'storm', 'voltage reduction', 'rain', 'flooding', 'generator trip', 'low flying helicopter', 'equipment failure', 'wind', 'hydro', 'nan', 'dust', 'public appeal', 'severe weather', 'fuel supply emergency', 'petroleum', 'vandalism'}


Looks like someone missed a tag here, let's add it for them:

In [312]:
df.loc[pd.isnull(df['Tags']),'Tags'] = 'wild fire'

In [317]:
df[df['Tags'].str.contains('vandalism')].count()

Event Description               346
Year                            346
Date Event Began                346
Time Event Began                  0
Date of Restoration             340
Time of Restoration               0
Respondent                      346
Geographic Areas                343
NERC Region                     346
Demand Loss (MW)                148
Number of Customers Affected    159
Tags                            346
dtype: int64